# Introduction

Find small single text files for **Language Modeling** experiements here ⬇️

https://www.kaggle.com/datasets/sovitrath/text-generation-language-modeling

In [2]:
import torch
import os
import numpy as np
import string
import re
import glob
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import tiktoken

from tqdm.auto import tqdm
from utils.text_gen import get_batch, train_step, val_step, NLPDataset
from collections import Counter
from torch.utils.data import DataLoader, Dataset, Subset
from torch.optim.lr_scheduler import StepLR

In [3]:
# Set seed.
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [4]:
!nvidia-smi

Sat Nov 11 09:21:46 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080        Off | 00000000:01:00.0  On |                  N/A |
|  0%   34C    P8              15W / 370W |    349MiB / 10240MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
CHECKPOINT_DIR = 'outputs/alice' 
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [9]:
dataset_dir = os.path.join('../../input', 'alice_short_story')
train_file = os.listdir(dataset_dir)
print(train_file)

['alice.txt']


## Prepare the Dataset
Let's find the longest review in the entire training set. As this will also contain the <br> tags, we will take the average of that.

We will pad the smaller sentences to this average length and truncate the longer sentences to the average length.

In [10]:
def find_longest_length(text_file_paths):
    """
    Find the longest sentence length in the entire training set. 

    :param text_file_paths: List, containing all the text file paths.

    Returns:
        max_len: Longest review length.
    """
    max_length = 0
    for path in text_file_paths:
        with open(path, 'r') as f:
            text = f.read()
            corpus = [
                word for word in text.split()
            ]
        if len(corpus) > max_length:
            max_length = len(corpus)
    return max_length


file_paths = []
file_paths.extend(glob.glob(os.path.join(
    dataset_dir, '*.txt'
)))
longest_sentence_length = find_longest_length(file_paths)
print(f"Total words (possibly, without tokenization): {longest_sentence_length} words")

Total words (possibly, without tokenization): 1243 words


In [11]:
# Number of words to generate in a sentence.
SEQUENCE_LENGTH = 128
NUM_WORDS = 50257  # Vocabulary size.

# Batch size.
BATCH_SIZE= 16
VALID_SPLIT = 0.1
MAX_ITERS = 5000

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### Tiktoken

In [13]:
enc = tiktoken.encoding_for_model("gpt2")
print(enc.n_vocab)

50257


## Prepare PyTorch Datasets and Data Loaders

In [14]:
file_paths = os.path.join(dataset_dir, train_file[0])

In [15]:
dataset_inst = NLPDataset(file_paths, enc)
dataset = dataset_inst.get_data()

In [16]:
print(f"Total tokens: {dataset.shape}")
print(f"Number of unique tokens: {len(np.unique(dataset))}")

Total tokens: torch.Size([1536])
Number of unique tokens: 618


In [17]:
dataset_size = len(dataset)
# Calculate the validation dataset size.
valid_size = int(VALID_SPLIT*dataset_size)
# Radomize the data indices.
indices = torch.randperm(len(dataset)).tolist()
# Training and validation sets.
dataset_train = dataset[:-valid_size]
dataset_valid = dataset[-valid_size:]

print(f"Number of training samples: {len(dataset_train)}")
print(f"Number of validation samples: {len(dataset_valid)}")

Number of training samples: 1383
Number of validation samples: 153


In [18]:
for i in range(100):
    src, tgt = get_batch(
        dataset_train, 
        sequence_length=SEQUENCE_LENGTH,
        batch_size=1
    )
    src_decoded = enc.decode_batch(src.numpy())
    tgt_decoded = enc.decode_batch(tgt.numpy())
    print('SRC')
    print(src_decoded)
    print('TGT')
    print(tgt_decoded)
    print('*'*50)

SRC
[' her dream adventures, Alice encountered the embodiment of her childhood innocence, a mischievous sprite who guided her through whimsical forests and sparkling meadows. She encountered her fears personified as towering monsters, which she had to face head-on and ultimately transform into allies.\n\nThrough her lucid dreaming exploration, Alice also stumbled upon a mysterious door, shimmering with golden light. Intrigued, she stepped through, finding herself in a realm where dreams and reality intertwined. Here, she met a group of Dream Weavers, ethereal beings tasked with shaping the fabric of dreams.\n\nThe Dream Weavers revealed to Alice that she']
TGT
[' dream adventures, Alice encountered the embodiment of her childhood innocence, a mischievous sprite who guided her through whimsical forests and sparkling meadows. She encountered her fears personified as towering monsters, which she had to face head-on and ultimately transform into allies.\n\nThrough her lucid dreaming explor

## Model

In [19]:
def generate_square_subsequent_mask(sz):
    """
    Generate a square mask for the sequence. The masked positions are filled with float('-inf').
    Unmasked positions are filled with float(0.0).
    """
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [20]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model, dropout=0.1):
        """
        :param max_len: Input length sequence.
        :param d_model: Embedding dimension.
        :param dropout: Dropout value (default=0.1)
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Inputs of forward function
        :param x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        """

        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [21]:
class TextGen(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads):
        super(TextGen, self).__init__()
        self.pos_encoder = PositionalEncoding(max_len=SEQUENCE_LENGTH, d_model=embed_dim)
        self.emb = nn.Embedding(vocab_size, embed_dim)
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_dim, 
            nhead=num_heads, 
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(
            decoder_layer=self.decoder_layer,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(embed_dim, vocab_size)
        self.dropout = nn.Dropout(0.2)
        
    # Positional encoding is required. Else the model does not learn.
    def forward(self, x):
        emb = self.emb(x)
        
        # Generate input sequence mask with shape (SEQUENCE_LENGTH, SEQUENCE_LENGTH)
        input_mask = generate_square_subsequent_mask(x.size(1)).to(x.device)
        
        x = self.pos_encoder(emb)
        x = self.decoder(x, memory=x, tgt_mask=input_mask, memory_mask=input_mask)
        x = self.dropout(x)
        out = self.linear(x)
        return out

In [22]:
model = TextGen(
    vocab_size=NUM_WORDS, 
    embed_dim=100,
    num_layers=1, 
    num_heads=1,
)

In [23]:
_ = model.to(device)

## Training

In [24]:
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(
    model.parameters(), 
    lr=0.0001,
)

# StepLR every specific number of epochs.
scheduler = StepLR(
    optimizer, 
    step_size=5, 
    gamma=0.5,
    verbose=True
)

# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.\n")

TextGen(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (emb): Embedding(50257, 100)
  (decoder_layer): TransformerDecoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
    (multihead_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
    (linear1): Linear(in_features=100, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=100, bias=True)
    (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm3): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
    (dropout3): Dropout(p=0.1, inplace=False)
  )
  (decoder): Transfor

In [25]:
# Lists to keep track of losses and accuracies.
train_loss, valid_loss = [], []
least_loss = float('inf')
# Start the training.
for iteration in range(MAX_ITERS):
    train_step_loss = train_step(
        model, 
        dataset_train, 
        optimizer, 
        criterion,
        SEQUENCE_LENGTH,
        NUM_WORDS,
        BATCH_SIZE,
        device
    )
    valid_step_loss = val_step(
        model, 
        dataset_valid,  
        criterion,
        SEQUENCE_LENGTH,
        NUM_WORDS,
        BATCH_SIZE,
        device
    )
    train_loss.append(train_step_loss.cpu().detach().numpy())
    valid_loss.append(valid_step_loss.cpu().detach().numpy())
    if iteration % 250 == 0:
        print(f"[INFO]: Iteration {iteration+1} of {MAX_ITERS}")
        print(f"Training loss: {train_step_loss}")
        print(f"Validation loss: {valid_step_loss}")
        print('-'*50)
        # Save model.
        if valid_step_loss < least_loss:
            print(f"Saving best model till now... LEAST LOSS {valid_step_loss:.3f}")
            least_loss = valid_step_loss
            torch.save(
                model, os.path.join(CHECKPOINT_DIR, 'model.pth')
            )
    #     if epoch + 1 <= 32:
#         scheduler.step()
torch.save(
    model, os.path.join(CHECKPOINT_DIR, 'final_model.pth')
)

[INFO]: Iteration 1 of 5000
Training loss: 11.087896347045898
Validation loss: 11.068445205688477
--------------------------------------------------
Saving best model till now... LEAST LOSS 11.068
[INFO]: Iteration 251 of 5000
Training loss: 6.13106107711792
Validation loss: 7.0298566818237305
--------------------------------------------------
Saving best model till now... LEAST LOSS 7.030
[INFO]: Iteration 501 of 5000
Training loss: 4.457359790802002
Validation loss: 6.470618724822998
--------------------------------------------------
Saving best model till now... LEAST LOSS 6.471
[INFO]: Iteration 751 of 5000
Training loss: 3.267477512359619
Validation loss: 6.290154457092285
--------------------------------------------------
Saving best model till now... LEAST LOSS 6.290
[INFO]: Iteration 1001 of 5000
Training loss: 2.351567268371582
Validation loss: 6.478583812713623
--------------------------------------------------
[INFO]: Iteration 1251 of 5000
Training loss: 1.8863301277160645


In [26]:
def save_plots(train_loss, valid_loss):
    """
    Function to save the loss and accuracy plots to disk.
    """
    plt.show()
    # Loss plots.
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_loss, color='blue', linestyle='-', 
        label='train loss'
    )
    plt.plot(
        valid_loss, color='red', linestyle='-', 
        label='validataion loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
#     plt.savefig(f"../outputs/loss.png")
    plt.show()

In [27]:
save_plots(train_loss, valid_loss)

## Test Model

In [28]:
trained_model = torch.load(
    os.path.join(CHECKPOINT_DIR, 'final_model.pth')
)

In [29]:
# test_loss = validate(
#     trained_model, 
#     dataset_test,  
#     criterion, 
#     device
# )

# print(f"Test loss: {test_loss}")

## Text Generation

In [30]:
def return_int_vector(enc, text):
        """
        Assign an integer to each word and return the integers in a list.
        """
        return enc.encode(text)

In [31]:
enc = tiktoken.encoding_for_model("gpt2")

In [36]:
def sample_next(predictions, temperature=1.0):
    """
    Implement variable-temperature sampling from a probability
    distribution.
    """
    # print(predictions.shape)
    predictions = predictions[:, -1, :] / temperature
    # print(predictions.shape)
    # print(torch.where(predictions > 0))
    probabilities = F.softmax(predictions, dim=-1).cpu()
    # print(probabilities.shape)
    next_token = torch.multinomial(probabilities, num_samples=1)
    # print(next_token)
    return int(next_token.cpu())

def text_generator(sentence, generate_length):
    trained_model.eval()
    temperatures = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0] 
    for temperature in temperatures:
        sample = sentence
        print(f"GENERATED SENTENCE WITH TEMPERATURE {temperature}")
        for i in range(generate_length):
            # print(f"Sample: {sample}")
            int_vector = return_int_vector(enc, sample)
            if len(int_vector) >= SEQUENCE_LENGTH - 1:
                break
            input_tensor = torch.tensor(int_vector, dtype=torch.long).to(device)  # Changed dtype to torch.long
            input_tensor = input_tensor.unsqueeze(0)
            with torch.no_grad():
                predictions = trained_model(input_tensor)
            next_token = sample_next(predictions, temperature)  # Pass the temperature to the sample function
            sample += enc.decode([next_token])
        print(sample)
        print('\n')

In [37]:
sentences = [
    "Alice was a curious and imaginative young girl"
]

In [38]:
generate_length = 16

In [39]:
for sentence in sentences:
    print(f"PROMPT: {sentence}")
    text_generator(sentence, generate_length)
    print('\n############\n')

PROMPT: Alice was a curious and imaginative young girl
GENERATED SENTENCE WITH TEMPERATURE 0.1
Alice was a curious and imaginative young girl who appeared and disappeared at will. The mischievous cat guided Alice to the


GENERATED SENTENCE WITH TEMPERATURE 0.2
Alice was a curious and imaginative young girl who appeared and disappeared at will. The mischievous cat guided Alice to the


GENERATED SENTENCE WITH TEMPERATURE 0.3
Alice was a curious and imaginative young girl who appeared and disappeared at will. The mischievous cat guided Alice to the


GENERATED SENTENCE WITH TEMPERATURE 0.4
Alice was a curious and imaginative young girl who appeared and disappeared at will. The mischievous cat guided Alice to the


GENERATED SENTENCE WITH TEMPERATURE 0.5
Alice was a curious and imaginative young girl who appeared and disappeared at will. The mischievous cat guided Alice to the


GENERATED SENTENCE WITH TEMPERATURE 0.6
Alice was a curious and imaginative young girl who appeared and disapp